In [5]:
import numpy as np
import cv2
import dlib
from scipy.spatial import distance
from pygame import mixer

In [9]:
detector=dlib.get_frontal_face_detector()
predictor=dlib.shape_predictor("shape_predictor_68_face_landmarks.dat")
    
drowsy=0
active=0
pause=1
songtoplay=["WakeUpSong.mp3"]

mixer.init()

def calc(eye):
    v1=distance.euclidean(eye[1],eye[5])
    v2=distance.euclidean(eye[2],eye[4])
    h=distance.euclidean(eye[0],eye[3])
    ratio=(v1+v2)/(2.0*h)
    if ratio<0.26:
        return 1
    else:
        return 0

def play_music():
    pause=0
    mixer.music.load(songtoplay[0])
    mixer.music.play()

cap=cv2.VideoCapture(0)

while True:
    success,img=cap.read()
    if success:
        gray=cv2.cvtColor(img,cv2.COLOR_BGR2GRAY)
        faces=detector(gray)
        for face in faces:
            x1 = face.left()
            y1 = face.top()
            x2 = face.right()
            y2 = face.bottom()
            cv2.rectangle(img,(x1,y1),(x2,y2),(255,255,0),2)
            
            landmark=predictor(gray,face)
            
            left=[]
            right=[]
            for n in range(36,42):
                x=landmark.part(n).x
                y=landmark.part(n).y
                left.append((x,y))
                
            for n in range(42,48):
                x=landmark.part(n).x
                y=landmark.part(n).y
                right.append((x,y))
    
            left_blink=calc(left)
            right_blink=calc(right)
            
            if left_blink==1 or right_blink==1:
                drowsy+=1
                active=0
                if drowsy>6:
                    cv2.putText(img,"DROWSY!! WAKE UP",(100,100),cv2.FONT_HERSHEY_SIMPLEX,1.4,(0,0,255),2)
                    if pause==0:
                        mixer.music.pause()
                        pause=1
                    
            else:
                active+=1
                drowsy=0
                if active>6:
                    cv2.putText(img,"ACTIVE",(100,100),cv2.FONT_HERSHEY_SIMPLEX,1.4,(0,128,0),2)
                    if pause==1:
                        play_music()
                      
        cv2.imshow("Driver Drowsiness Detection FRAME",img)
        if cv2.waitKey(1) & 0xFF == ord('q'):
            mixer.music.pause()
            break
            
cap.release()
cv2.destroyAllWindows()                  